In [30]:
library(dplyr)
library(lme4)
library(car) # for VIF calculation
library(tidyverse)
library(boot)
library(modelsummary)
library(lmerTest)
library(ggeffects)
library(magrittr)
library(broom)
library(broom.mixed)
library(sjPlot)
library(sjmisc)
library(sjlabelled)
library(jtools)
set.seed(12696921)

CORONA_INTERIM_PATH <- "/m/cs/work/luongn1/digirhythm/data/interim/corona/"
CORONA_PROCESSED_PATH <- "/m/cs/work/luongn1/digirhythm/data/processed/corona/"
SIMILARITY_PATH <- "/m/cs/work/luongn1/digirhythm/data/processed/corona/similarity_matrix/"

# Read survey data
survey <- read.csv(paste0(CORONA_INTERIM_PATH, "survey_all.csv"))

# Filter out 'non-binary' gender
survey <- survey %>% filter(gender != 'non-binary')

# Read similarity data
sim_baseline <- read.csv(paste0(SIMILARITY_PATH, "si/similarity_baseline_4epochs.csv"), row.names = 1)

# Keep only necessary columns
IVs <- c("subject_id", "age", "gender", "occupation", "origin", "children_at_home", "BIG5_Extraversion", "BIG5_Agreeableness", "BIG5_Conscientiousness", "BIG5_Neuroticism", "BIG5_Openness", "MEQ")
demographics_df <- survey %>% select(all_of(IVs)) %>% drop_na()

# Calculate average similarity
avg_sim_baseline <- rowMeans(sim_baseline, na.rm = TRUE)
avg_sim_baseline <- data.frame(subject_id = rownames(sim_baseline), DV = avg_sim_baseline)

# Merge datasets
dataset <- merge(avg_sim_baseline, demographics_df, by = 'subject_id', all.x = TRUE)

# Define a function to extract the coefficients
boot_fn <- function(data, indices) {
  d <- data[indices, ]  # Extract the bootstrapped sample
  fit <- lm(DV ~ age + origin + occupation + children_at_home + MEQ, data = d)
  return(coef(fit))
}

# Regression analysis with bootstrapping
regression_analysis <- function(df, y, X) {
  df <- df %>% drop_na()
  model <- lm(as.formula(paste(y, "~", paste(X, collapse = "+"))), data = df)
  vif_values <- vif(model)
summ(model, scale=TRUE, vifs=TRUE, confint = TRUE, digits = 3)
}

# Run the analysis
regression_analysis(dataset, "DV", c("age", "origin", "gender", "occupation", "children_at_home", "MEQ"))


MODEL INFO:
Observations: 115
Dependent Variable: DV
Type: OLS linear regression 

MODEL FIT:
F(6,108) = 3.198, p = 0.006
R² = 0.151
Adj. R² = 0.104 

Standard errors: OLS
---------------------------------------------------------------------------
                           Est.     2.5%    97.5%    t val.       p     VIF
---------------------- -------- -------- -------- --------- ------- -------
(Intercept)               0.680    0.669    0.690   125.246   0.000        
age                       0.005    0.000    0.010     2.018   0.046   1.059
origin                   -0.017   -0.030   -0.004    -2.509   0.014   1.415
gender1                   0.005   -0.006    0.016     0.960   0.339   1.073
occupation                0.014    0.002    0.027     2.362   0.020   1.482
children_at_home          0.002   -0.003    0.007     0.934   0.352   1.016
MEQ                       0.007    0.002    0.012     2.633   0.010   1.033
--------------------------------------------------------------------

In [31]:
# Read survey data
data <- read.csv(paste0(CORONA_PROCESSED_PATH, "14ds_regularity_wellbeing.csv"))

# Fitting the model for y1 with a random intercept for 'subject'
# Define the formula
formula1 <- 'PHQ ~ 1 + 
baseline_similarity +
steps.night.14ds.sum.norm + steps.morning.14ds.sum.norm + steps.afternoon.14ds.sum.norm + steps.evening.14ds.sum.norm +
  steps.total.norm +
  tst.norm.mean + 
  midsleep.norm.mean + 
  heart_rate_variability_avg.mean.norm + 
age.norm + gender + occupation + origin +
  (1|subject_id)'

formula2 <- 'PSS ~ 1 + 
baseline_similarity +
steps.night.14ds.sum.norm + steps.morning.14ds.sum.norm + steps.afternoon.14ds.sum.norm + steps.evening.14ds.sum.norm +
  steps.total.norm +
  tst.norm.mean + 
  midsleep.norm.mean + 
  heart_rate_variability_avg.mean.norm + 
age.norm + gender + occupation + origin +
  (1|subject_id)'
formula3 <- 'PSQI ~ 1 + 
baseline_similarity +
steps.night.14ds.sum.norm + steps.morning.14ds.sum.norm + steps.afternoon.14ds.sum.norm + steps.evening.14ds.sum.norm +
  steps.total.norm +
  tst.norm.mean + 
  midsleep.norm.mean + 
  heart_rate_variability_avg.mean.norm + 
age.norm + gender + occupation  + origin +
  (1|subject_id)'


# Fit the linear mixed-effects model
fit1 <- lmer(formula1, data = data)
fit2 <- lmer(formula2, data = data)
fit3 <- lmer(formula3, data = data)

# Display the summary of the model fit
tab_model(fit1, fit2, fit3,
         show.r2 = TRUE,
    show.icc = FALSE,
    show.re.var = FALSE,
    emph.p = TRUE,
    file = "temp.html")

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



In [32]:
labels = rev(c("Baseline similarity", "Steps:Night", "Steps:Morning", 
           "Steps:Afternoon", "Steps:Evening", "Total steps", "Total sleep time",
           "Midsleep", "HRV", "Age", "Gender", "Occupation", "Origin"))

fplot <- plot_models(
  fit1, fit2, fit3,
  axis.labels = labels,
  m.labels = c("PHQ", "PSS", "PSQI"),
     p.shape = TRUE
)
save_plot('forestplot.png', fplot)

png 
  2

In [46]:
pearsonboot = function(var1, data, indices,...) # indices needs to be supplied to boot function
{
  d1 = data[var1][indices,]
  d2 = data[indices,]

  pear = cor(x = d1, y = d2, use = "pairwise.complete.obs", method = "pearson")
  pear = pear[2:length(pear)]

  return(pear)
}

cormat = data.frame(data$PSS, data$PHQ, data$PSQI, data$baseline_similarity)
boots = boot(data = cormat[c("data.PSS", "data.baseline_similarity")], pearsonboot, R = 1000, var1 = "data.PSS")
bootsCI = boot.ci(boots, type = "perc")
bootsCI = round(c(bootsCI$t0, bootsCI$percent[4],bootsCI$percent[5]), digits = 2)
bootsCI = paste0("$r$ = ", bootsCI[1], ", $CI_{95\\%}$ = [", bootsCI[2], ", ", bootsCI[3], "]")
EES = gsub("0*\\.",".", bootsCI)
EES

[1] "$r$ = -.06, $CI_{95\\%}$ = [-.13, .01]"

In [40]:
colnames(cormat)

[1] "data.PSS"                 "data.PHQ"                
[3] "data.PSQI"                "data.baseline_similarity"